In [1]:
import numpy as np
import cv2
import sys
import os
import pickle
import pandas as pd

from src import RESULT_DIR
from src.utils import MCC, DIS


# Directories
output_dir = os.path.join(RESULT_DIR, 'test_results')
tables_dir = os.path.join(output_dir, 'tables')

In [25]:
def get_table(dbase, level, ground_truth):
    results_csv = os.path.join(tables_dir, 
                               '{0}_full_results.csv'.format(dbase))

    # Columns
    columns = [k + '_{0}{1}'.format(level[0], ground_truth[0]) 
            for k in ['gt', 'tn', 'fp', 'fn', 'tp']]
    all_columns = ['video', 'frame', 'fold'] + columns
    cc = all_columns[4:]
    
    res_df = pd.read_csv(results_csv, index_col=0)
    res_df = res_df[all_columns]
    res_df = res_df.groupby([vid_fold]).sum().loc[:, cc]
    res_df['MCC'] = res_df.apply(lambda row: MCC(row[cc[0]], row[cc[1]],
                                                row[cc[2]], row[cc[3]]),
                                 axis=1)
    res_df['DIS'] = res_df.apply(lambda row: DIS(row[cc[0]], row[cc[1]],
                                                row[cc[2]], row[cc[3]]),
                                 axis=1)
    sum_df = res_df.sum()
    res_df.loc['Average', :] = res_df.mean()
    res_df.loc['Std. deviation', :] = res_df.std()
    res_df.loc['Overall', 'MCC'] = MCC(sum_df[cc[0]], sum_df[cc[1]],
                                    sum_df[cc[2]], sum_df[cc[3]])
    res_df.loc['Median', :] = res_df.median()
    res_df.loc['Overall', 'DIS'] = DIS(sum_df[cc[0]], sum_df[cc[1]],
                                    sum_df[cc[2]], sum_df[cc[3]])
    res_df = res_df[['MCC', 'DIS']]
    return res_df


#res_df = get_table(dbase, level, ground_truth)

In [29]:
dbase = 'LightGBM_warp'
level = 'frame'
ground_truth = 'silhouette'
vid_fold = 'video'

dbases = ['DAOMC', 'ADMULT', 'MCBS_Both', 'mcDTSR', 'TCF_LMO',
          'LightGBM_temporal','LightGBM_warp']


results = {d:get_table(d, level, ground_truth) for d in dbases}


# concatenate the DataFrames horizontally
df_concat = pd.concat(results.values(), axis=1)

# create a multi-level column index using the dictionary keys
columns = pd.MultiIndex.from_tuples([(key, col) for key in results.keys()
                                     for col in results[key].columns])

# assign the multi-level column index to the concatenated DataFrame
df_concat.columns = columns
df_concat = df_concat.loc[df_concat.index.isin(['Average', 'Std. deviation', 
                                                'Median', 'Overall']), :]

In [30]:
df_mcc = df_concat.loc[:, (slice(None), 'MCC')]
df_dis = df_concat.loc[:, (slice(None), 'DIS')]

df_mcc = df_mcc.droplevel(level=1, axis=1)
df_dis = df_dis.droplevel(level=1, axis=1)


In [31]:
df_mcc

,DAOMC,ADMULT,MCBS_Both,mcDTSR,TCF_LMO,LightGBM_temporal,LightGBM_warp
video,,,,,,,
Average,0.284018,0.353208,0.002811,0.0,0.223498,0.007678,0.546110
Std. deviation,0.334987,0.425471,0.064410,0.0,0.323609,0.035648,0.406139
Overall,0.327038,0.388037,0.003524,0.0,0.359926,0.036067,0.638534
Median,0.285082,0.440322,0.000000,0.0,0.000000,0.000000,0.740274


In [23]:
print(df_dis.to_latex())

\begin{tabular}{lrrrrrrr}
\toprule
{} &     DAOMC &    ADMULT &  MCBS\_Both &    mcDTSR &   TCF\_LMO &  LightGBM\_temporal &  LightGBM\_warp \\
video          &           &           &            &           &           &                    &                \\
\midrule
Average        &  0.569579 &  0.715649 &   1.310878 &  1.046858 &  0.840063 &           1.006345 &       0.401983 \\
Std. deviation &  0.350320 &  0.305444 &   0.132755 &  0.097670 &  0.279312 &           0.029174 &       0.401205 \\
Overall        &  0.538009 &  0.720537 &   1.267981 &  1.013964 &  0.907302 &           0.999923 &       0.374390 \\
Median         &  0.538876 &  0.768262 &   1.366085 &  1.000459 &  1.000000 &           1.000000 &       0.188889 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_4882/1507108909.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_dis.to_latex())


In [72]:
# Create the style object
df_style = df_mcc.style.apply(
    lambda x: ['font-weight: bold' if v == x[x.index.max()] else '' for v in x],
    axis=1
).apply(
    lambda x: ['font-weight: bold' if v == x[x.index.min()] else '' for v in x],
    axis=1, subset=pd.IndexSlice['Std. deviation', :]
)

# Convert to LaTeX
latex_table = df_style.to_latex()
print(latex_table)


\begin{tabular}{lrrrrrrr}
 & DAOMC & ADMULT & MCBS_Both & mcDTSR & TCF_LMO & LightGBM_temporal & LightGBM_warp \\
video &  &  &  &  &  &  &  \\
Average & 0.380501 & 0.457119 & 0.000558 & \font-weightbold -0.014802 & 0.030048 & 0.010230 & 0.623659 \\
Std. deviation & 0.283957 & \font-weightbold 0.201435 & 0.027383 & \font-weightbold 0.043969 & 0.085761 & 0.045464 & 0.270707 \\
Overall & 0.604219 & 0.584970 & 0.006895 & \font-weightbold -0.014536 & 0.064795 & 0.052447 & 0.772486 \\
Median & 0.460846 & 0.477773 & -0.007479 & \font-weightbold 0.000000 & \font-weightbold 0.000000 & \font-weightbold 0.000000 & 0.721401 \\
\end{tabular}



In [73]:
# Create the style object
df_style = df_dis.style.apply(
    lambda x: ['font-weight: bold' if v == x[x.index.min()] else '' for v in x],
    axis=1
).apply(
    lambda x: ['font-weight: bold' if v == x[x.index.min()] else '' for v in x],
    axis=1, subset=pd.IndexSlice['Std. deviation', :]
)
# Convert to LaTeX
latex_table = df_style.to_latex()
print(latex_table)

\begin{tabular}{lrrrrrrr}
 & DAOMC & ADMULT & MCBS_Both & mcDTSR & TCF_LMO & LightGBM_temporal & LightGBM_warp \\
video &  &  &  &  &  &  &  \\
Average & 0.678599 & \font-weightbold 0.651834 & 0.985454 & 0.981277 & 0.977394 & 0.982307 & 0.461805 \\
Std. deviation & 0.271239 & \font-weightbold \font-weightbold 0.192956 & 0.024443 & 0.160750 & 0.050829 & 0.047462 & 0.287643 \\
Overall & 0.506412 & \font-weightbold 0.534708 & 0.979873 & 0.735264 & 0.906308 & 0.896709 & 0.319107 \\
Median & 0.623718 & \font-weightbold 0.667969 & 0.995495 & 1.069551 & 1.000048 & 1.000371 & 0.383795 \\
\end{tabular}



In [68]:
# create the styler object
s = df_mcc.style

# apply the function to each cell using applymap()
s.applymap(highlight_greatest)

# render the styled dataframe as a LaTeX table
s.to_latex()

AttributeError: 'float' object has no attribute 'index'